In [ ]:
import pandas as pd
import numpy as np
import csv
from mlxtend.frequent_patterns import apriori, association_rules
import matplotlib.pyplot as plt
#from anytree import NodeMixin, RenderTree
#from anytree.exporter import UniqueDotExporter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# Importing Data
data = pd.read_csv("/content/drive/MyDrive/ml-latest-small/ratings.csv")
data.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [ ]:
data.drop('timestamp',axis=1,inplace=True)
data.head()

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0
3,1,47,5.0
4,1,50,5.0


In [ ]:
basket = (data.groupby(['userId','movieId'])['rating'].sum().unstack().fillna(0))
basket.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,4.0,0.0,4.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.0,4.0,0.0,3.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
def change_into_encode(x):
  if x>=2.5:
    return 1
  else:
    return 0

In [ ]:
basket_sets = basket.applymap(lambda x:x>=2.5)
basket_sets.head()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
5,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
basket_sets.tail()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
606,True,False,False,False,False,False,True,False,False,False,True,False,False,False,True,False,True,True,False,False,False,False,False,False,False,False,False,True,True,False,False,True,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
607,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,True,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
608,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,True,True,False,False,True,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
609,True,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
610,True,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False


In [ ]:
basket_sets.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 610 entries, 1 to 610
Columns: 9724 entries, 1 to 193609
dtypes: bool(9724)
memory usage: 5.7 MB


In [ ]:
basket_sets.describe()

movieId,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,34,36,38,39,40,41,42,43,...,185135,185435,185473,185585,186587,187031,187541,187593,187595,187717,188189,188301,188675,188751,188797,188833,189043,189111,189333,189381,189547,189713,190183,190207,190209,190213,190215,190219,190221,191005,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
count,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,...,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610,610
unique,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,2,1,2,2,1,2,1,1,1,1,2,2,2,2,2,2,2,2,2,2,2
top,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
freq,403,509,565,606,570,511,564,604,597,490,544,600,603,592,602,532,549,592,560,603,529,578,598,588,545,597,603,599,575,608,578,437,497,547,608,521,608,595,605,603,...,608,609,609,608,609,608,607,599,605,609,609,607,609,609,609,609,609,609,608,609,610,609,609,610,609,610,610,610,610,609,609,609,609,609,609,609,609,609,609,609


In [ ]:
freq_items = apriori(basket_sets,min_support=0.1,use_colnames=True)
freq_items.head(11)

,support,itemsets
0,0.339344,(1)
1,0.165574,(2)
2,0.162295,(6)
3,0.196721,(10)
4,0.108197,(11)
5,0.127869,(16)
6,0.100000,(17)
7,0.132787,(21)
8,0.106557,(25)
9,0.283607,(32)


Question - 2) Using the association rule mining techniques, build a recommendation system for the
dataset provided to you. Your system should be able to recommend movies given a customer profile. i.e.
what set of movies a customer has watched in the past? The set can contain any non-zero number of
movies. Let us say a customer has seen "avengers, iron man", the system can recommend the following
movies "captain america, hulk, thor, doctor strange". The final scores will be computed using the metric
"Precision@k". For a given customer profile, you will recommend four best movies, i.e. k=4.

In [ ]:
rules = association_rules(freq_items,metric='lift',min_threshold=0.05)
rules.head(11)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,(1),(2),0.339344,0.165574,0.100000,0.294686,1.779787,0.043813,1.183056
1,(2),(1),0.165574,0.339344,0.100000,0.603960,1.779787,0.043813,1.668156
2,(1),(10),0.339344,0.196721,0.101639,0.299517,1.522544,0.034883,1.146750
3,(10),(1),0.196721,0.339344,0.101639,0.516667,1.522544,0.034883,1.366874
4,(32),(1),0.283607,0.339344,0.159016,0.560694,1.652286,0.062776,1.503861
5,(1),(32),0.339344,0.283607,0.159016,0.468599,1.652286,0.062776,1.348122
6,(1),(34),0.339344,0.185246,0.126230,0.371981,2.008037,0.063367,1.297339
7,(34),(1),0.185246,0.339344,0.126230,0.681416,2.008037,0.063367,2.073725
8,(1),(47),0.339344,0.313115,0.144262,0.425121,1.357716,0.038009,1.194834
9,(47),(1),0.313115,0.339344,0.144262,0.460733,1.357716,0.038009,1.225099


In [ ]:
rules.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 552720 entries, 0 to 552719
Data columns (total 9 columns):
 #   Column              Non-Null Count   Dtype  
---  ------              --------------   -----  
 0   antecedents         552720 non-null  object 
 1   consequents         552720 non-null  object 
 2   antecedent support  552720 non-null  float64
 3   consequent support  552720 non-null  float64
 4   support             552720 non-null  float64
 5   confidence          552720 non-null  float64
 6   lift                552720 non-null  float64
 7   leverage            552720 non-null  float64
 8   conviction          552720 non-null  float64
dtypes: float64(7), object(2)
memory usage: 38.0+ MB


In [ ]:
rules.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,552720.000000,552720.000000,552720.000000,552720.000000,552720.000000,552720.000000,5.527200e+05
mean,0.213629,0.213629,0.108774,0.593582,2.952123,0.068132,inf
std,0.097304,0.097304,0.011431,0.207206,0.875496,0.012473,NaN
min,0.100000,0.100000,0.100000,0.191824,0.945248,-0.005982,9.598991e-01
25%,0.142623,0.142623,0.101639,0.423841,2.271719,0.060656,1.458230e+00
50%,0.181967,0.181967,0.104918,0.596330,2.873758,0.069788,1.980635e+00
75%,0.257377,0.257377,0.111475,0.756098,3.532819,0.076619,3.011547e+00
max,0.521311,0.521311,0.365574,1.000000,7.743653,0.169874,inf


In [ ]:
rules_modified = rules[(rules['lift'] >= 1.5) & (rules['confidence'] >= 0.60)]

In [ ]:
rules_modified.describe()

,antecedent support,consequent support,support,confidence,lift,leverage,conviction
count,271633.000000,271633.000000,271633.000000,271633.000000,271633.000000,271633.000000,2.716330e+05
mean,0.143813,0.269948,0.108679,0.770723,3.161499,0.070657,inf
std,0.024962,0.101085,0.011189,0.108931,0.943862,0.012444,NaN
min,0.100000,0.103279,0.100000,0.600000,1.500161,0.033340,1.520492e+00
25%,0.124590,0.188525,0.101639,0.677419,2.421610,0.063257,2.400240e+00
50%,0.140984,0.247541,0.104918,0.759036,3.108671,0.072464,3.043033e+00
75%,0.159016,0.326230,0.111475,0.857143,3.828702,0.078866,4.719523e+00
max,0.475410,0.521311,0.321311,1.000000,7.743653,0.169874,inf


In [ ]:
rules_modified.head()

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
1,(2),(1),0.165574,0.339344,0.100000,0.603960,1.779787,0.043813,1.668156
7,(34),(1),0.185246,0.339344,0.126230,0.681416,2.008037,0.063367,2.073725
18,(1),(260),0.339344,0.391803,0.204918,0.603865,1.541245,0.071962,1.535326
29,(364),(1),0.272131,0.339344,0.167213,0.614458,1.810721,0.074867,1.713576
31,(367),(1),0.214754,0.339344,0.129508,0.603053,1.777114,0.056633,1.664344


In [ ]:
data_movies = pd.read_csv("/content/drive/MyDrive/ml-latest-small/movies.csv")
data_movies.head()

,movieId,title,genres
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy
1,2,Jumanji (1995),Adventure|Children|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama|Romance
4,5,Father of the Bride Part II (1995),Comedy


In [ ]:
def question_2_func(print_movie_name,movie_name,tsv_writer):
  movie_id = list(data_movies.loc[data_movies['title']==movie_name,'movieId'])
  #movie_id[0]

  try:
    recommend = list(rules_modified.loc[rules_modified['antecedents'] == frozenset({movie_id[0]}), 'consequents'])
  except:
    recommend = []
  recommend_list = []
  for nik in recommend:
    recommend_list.extend(list(nik))
  recommend_list = list(set(recommend_list))
  #recommend_list

  recommend_list_modified = []
  for itr in range(len(recommend_list)):
    val = np.mean(data.loc[data['movieId']==recommend_list[itr], 'rating'])
    recommend_list_modified.append([recommend_list[itr], val])
  recommend_list_modified.sort(key=lambda x:x[1])
  recommend_list_modified = recommend_list_modified[:4]
  #recommend_list_modified

  list_input_in_file = []
  #con2 = "{} \n {}".format(con2, print_movie_name)
  #list_input_in_file.append(print_movie_name)

  con2 = ""
  for tmp in print_movie_name:
    con2 = "{} \n {}".format(con2, tmp)

  list_input_in_file.append(con2)

  con = ""
  for tmp in recommend_list_modified:
    movie_name_suggested = list(data_movies.loc[data_movies['movieId']==tmp[0],'title'])
    con = "{} \n {}".format(con, movie_name_suggested[0])
    #print(con)

  list_input_in_file.append(con)
  tsv_writer.writerow(list_input_in_file)


output_file = "/content/drive/MyDrive/ml-latest-small/output.tsv"

with open(output_file, 'wt') as out_file:
    tsv_writer = csv.writer(out_file,delimiter='\t')
    tsv_writer.writerow(["movies","recommendation"])

    tsv_reader = open("X_test_multiX4.tsv", 'r')

    data2 = csv.reader(tsv_reader, delimiter="\t")
    count = 0
    for row in data2:
      if count != 0:
        print(row)
        movie_name = row
        first_name = row[0].split('\n')
        print(first_name[0])
        question_2_func(first_name,first_name[0],tsv_writer)
      count += 1
    # movie_name = "Star Wars: Episode V - The Empire Strikes Back (1980)"
    # question_2_func(movie_name,tsv_writer)

    # movie_name5 = "Harry Potter and the Chamber of Secrets (2002)"
    # question_2_func(movie_name5,tsv_writer)

    # movie_name4 = "Lord of the Rings: The Fellowship of the Ring, The (2001)"
    # question_2_func(movie_name4,tsv_writer)


['Jurassic Park (1993)\nFirst Kid (1996)\n101 Dalmatians (1996)\nHomeward Bound: The Incredible Journey (1993)', '']
Jurassic Park (1993)
['Liar Liar (1997)\nAnastasia (1997)\nMurder at 1600 (1997)\nConspiracy Theory (1997)', '']
Liar Liar (1997)
['Clockwork Orange, A (1971)\nPostino, Il (1994)\nChasing Amy (1997)\nPiano, The (1993)', '']
Clockwork Orange, A (1971)
['Toy Story (1995)\n Pretty Woman (1990)', '']
Toy Story (1995)
["Hoop Dreams (1994)\nOne Flew Over the Cuckoo's Nest (1975)\nSnow White and the Seven Dwarfs (1937)\nTaxi Driver (1976)", '']
Hoop Dreams (1994)
['Raising Arizona (1987)\nApocalypse Now (1979)\nAnnie Hall (1977)\nSecrets & Lies (1996)', '']
Raising Arizona (1987)
['Some Like It Hot (1959)\nCop Land (1997)\nThis Is Spinal Tap (1984)\nRaging Bull (1980)', '']
Some Like It Hot (1959)
["Titanic (1997)\nRosewood (1997)\nSoul Food (1997)\nEve's Bayou (1997)", '']
Titanic (1997)
['Toy Story (1995)\nStar Wars (1977)\nGodfather, The (1972)\nWilly Wonka and the Chocolate

In [ ]:
# tsv_reader = open("X_test_multiX4.tsv", 'r')
# data = csv.reader(tsv_reader, delimiter="\t")
# count = 0
# for row in data:
#   if count != 0:
#     print(row)
#     movie_name = row
#     first_name = row[0].split('\n')
#     print(first_name[0])
#     question_2_func(first_name[0],tsv_writer)
#   count += 1

Question - 3) Visualize the maximal frequent pattern set.

In [ ]:
# unique_support = freq_items.support.unique()

In [ ]:
# dic_same_support = {}
# for tmp in unique_support:
#   val = list(freq_items.loc[freq_items.support == tmp]['itemsets'])
#   dic_same_support[tmp] = val
# #dic_same_support

In [ ]:
# dic_less_support = {}
# for i in range(len(unique_support)):
#   val = list(freq_items.loc[freq_items.support <= unique_support[i]]['itemsets'])
#   dic_less_support[unique_support[i]] = val
# #dic_less_support

In [ ]:
# max_frequent_itemset = []
# for i,j in freq_items.iterrows():
#   close_bool = True
#   cli = j['itemsets']
#   cls = j['support']
#   check_for_set = dic_less_support[cls]
#   for itr in check_for_set:
#     if cli!=itr:
#       if frozenset.issubset(cli,itr):
#         close_bool = False
#         break
#   if close_bool == True:
#     max_frequent_itemset.append(j['itemsets'])

# max_frequent_itemset